# OliktokPoint_Smith method for calculating empirical transfer function and calculate waves
1. Calculate PSD of strain from DAS. Divide by SWIFT PSD corrected for depth attenuation to approximate expected seafloor pressure . 
2.Take median of all correction factors (temporal) to be used as channel-specific correction factor. Does the shape make sense with analytical transfer function?  
3. For each timestep, calculate PSD of raw DAS strain, then multiply by correction factor and divide by attenuation correction.
4. For each timestep, calculate bulk wave parameters for both DAS and SWIFT over the wavelengths 0.06 to 0.4 Hz (can change this if desired?)

In [2]:
import glob
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from scipy import signal

from datetime import date, timedelta, datetime
import matplotlib.dates as mdates
plt.rcParams.update({'font.size': 12})
from pathlib import Path

from scipy.signal import argrelextrema

import scipy.io
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# read in global parameters (das info and SWIFT )
das_info = pd.read_csv('OliktokPoint_Smith/CODAS_info.csv')
SWIFT = xr.open_dataset('OliktokPoint_Smith/SWIFT18_OliktokPt_Aug2022_reprocessedIMU_1Hzspectra_convertedTe.nc')

In [13]:
#das data (in future, loop thru channens)

#note: channel 393 is closest to the SWIFT location
for ci, channel in enumerate(das_info.Channel[393:394]):#393-400
    #channel = 7960
    print('processing channel ' + str(channel)+', '+str(datetime.now()))
    das_info_channel = das_info[das_info['Channel']==channel]

    DAS_dir ='/Users/msmith/Documents/DAS/CODAS/uw_2022_08/'+str(channel)+'/'
    das_times = sorted([datetime.strptime(x.split('/')[-1][10:25],'%Y%m%d_%H%M%S') for x in glob.glob(DAS_dir+'/*.ncdf')])
    das_timestr = sorted([x.split('/')[-1][10:25] for x in glob.glob(DAS_dir+'/*.ncdf')])

    #calculate depth attenuation function for this channel location
    depth = -das_info_channel['Water Depth'].values
    k = (2*np.pi*SWIFT.freq.values)**2 / 9.8
    attenuation = np.exp(k*depth)
    attenuation = attenuation**2; # square for energy 

    #plot directories (base and channel speific)
    plotdir = '/Users/msmith/Documents/DAS/CODAS/202208_reprocessing/'
    Path(plotdir+str(channel)).mkdir(parents=True, exist_ok=True)
    plotdir_chan = plotdir+str(channel)+'/'
    
    #first need to save array of correlation functions for all times (so don't have to re-calc in each loop)

    corr_factor = []
    for ti,das_time in enumerate(das_times):
        das_file = glob.glob(DAS_dir +  'CODAS.D*__' + das_timestr[ti] + '.*__chn*'+str(channel)+'.ncdf')[0]
        ds_disk = xr.open_dataset(das_file)

        #spectra of DAS, interpolate to SWIFT frequency
        window = 128
        nfft = 256
        f_psd, ds_psd = signal.welch((ds_disk.data),fs=2,nfft=nfft,nperseg=window)

        from scipy.interpolate import interp1d
        ds_psd = interp1d(f_psd, ds_psd)(SWIFT.freq)

        #calculate equivalent attenuated SWIFT spectra
        SWIFT_E_att = SWIFT.sel(time=das_time,method='nearest').energy*attenuation

        if das_time == datetime(2022,8,17,18,0,9):
            fig, ax = plt.subplots(1,2,figsize=(12,4))
            ax[0].loglog(SWIFT.freq,SWIFT_E_att,color='orange')
            ax[0].loglog(SWIFT.freq,ds_psd,color='purple')
            ax[0].legend(['SWIFT (attenuated to depth)','strain'])
            ax[0].set_ylabel('PSD')
            ax[0].set_xlabel('freq [Hz]')
            ax[0].grid(True)

            ax[1].loglog(SWIFT.freq,SWIFT_E_att/ds_psd,color='purple')
            ax[1].set_ylabel('correction factor (~pressure/strain)')
            ax[1].set_xlabel('freq [Hz]')
            ax[1].grid(True)
            #plt.xlim((0.05,0.3))

            fig.suptitle('channel '+str(channel)+', time ' +das_timestr[ti])

            plt.savefig(plotdir_chan + 'WaveCorrectionFactorCalc_channel'+str(channel)+'_'+das_timestr[ti]+'_1Hz.jpg',bbox_inches='tight',dpi=110)
            plt.close(fig)
        """
        """

        corr_factor.append((SWIFT_E_att/ds_psd).values)
    corr_factor = np.array(corr_factor) 

    #save nanmedian correction factor 
    pd.DataFrame(data=np.transpose([SWIFT.freq.values,np.nanmedian(corr_factor,0)]),  columns=['freq','corr_factor']).to_csv(plotdir+'SpectralCorrectionFactor_channel'+str(channel)+'_1Hz.csv') 

    
    #plot and save all empirical correction factors
    fig, ax = plt.subplots(figsize=(6,4))

    for ti,das_time in enumerate(das_times):
        ax.loglog(SWIFT.freq,np.array(corr_factor)[ti,:],'b',alpha=.3,color='purple')
    ax.loglog(SWIFT.freq,np.nanmedian(corr_factor,0),'k',linewidth=2)
    ax.set_xlabel('freq [Hz]')
    ax.set_ylabel('correction factor (~pressure/strain)')
    ax.grid(True)
    
    ax.set_title('channel '+str(channel))

    plt.savefig(plotdir + 'WaveCorrectionFactor_all_channel'+str(channel)+'.jpg',bbox_inches='tight',dpi=110)
    plt.close(fig)

processing channel 7960, 2023-04-19 12:48:51.457257
